<a href="https://colab.research.google.com/github/keywoong/deeplearning_with_python/blob/main/7_Advanced_deep_learning_best_practices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from keras import Input, layers

input_tensor = Input(shape = (32,))
# 텐서 타입의 입력값
dense = layers.Dense(32, activation = 'relu')
# 이 층이 함수로써 작용하는 레이어이다.
output_tensor = dense(input_tensor)
# 텐서 타입의 출력값

In [ ]:
from keras.models import Sequential, Model
from keras import layers
from keras import Input

seq_model = Sequential()
seq_model.add(layers.Dense(32, activation = 'relu', input_shape = (64,)))
seq_model.add(layers.Dense(32, activation = 'relu'))
seq_model.add(layers.Dense(10 ,activation = 'softmax'))
# 이러한 구조가 바로 Sequential 방식의 모델 구성이다.

unrelated_input = Input(shape = (64,))
x = layers.Dense(32, activation = 'relu')(unrelated_input)
x = layers.Dense(32, activation = 'relu')(x)
output_tensor = layers.Dense(10 ,activation = 'softmax')(x)
# 이러한 구조가 바로 function API 방식의 모델 구성이다.

model = Model(unrelated_input, output_tensor)
# function API 모델은 텐서 데이터를 입력받아 텐서 데이터를 출력한다.

model.summary()

Model: "functional_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 64)]              0         
_________________________________________________________________
dense_22 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_23 (Dense)             (None, 32)                1056      
_________________________________________________________________
dense_24 (Dense)             (None, 10)                330       
Total params: 3,466
Trainable params: 3,466
Non-trainable params: 0
_________________________________________________________________


In [ ]:
unrelated_input = Input(shape = (64,))
bad_model = model = Model(unrelated_input, output_tensor)

ValueError: Graph disconnected: cannot obtain value for tensor Tensor("input_2:0", shape=(None, 64), dtype=float32) at layer "dense_4". The following previous layers were accessed without issue: []

## 7.1 Functional API implementation of a two-input question-answering model

In [ ]:
from keras.models import Model
from keras import layers
from keras import Input

text_vocabulary_size = 10000
question_vocabulary_size = 10000
answer_vocabulary_size = 500

text_input = Input(shape = (None,), dtype = 'int32', name = 'text')
# text_input은 정수형태의 시퀀스이다. name은 선택사항.

embedded_text = layers.Embedding(64, text_vocabulary_size)(text_input)
# text_input을 사이즈가 64인 형태의 벡터로 바꾸어준다.
# tf.Embedding
'''
tf.keras.layers.Embedding(
    input_dim, output_dim, embeddings_initializer='uniform',
    embeddings_regularizer=None, activity_regularizer=None,
    embeddings_constraint=None, mask_zero=False, input_length=None, **kwargs
)
>> 양의 정수 (인덱스)를 고정 된 크기의 조밀 한 벡터로 변환합니다.
'''

encoded_text = layers.LSTM(32)(embedded_text)
# LSTM을 통해 유닛이 32인 벡터로 인코딩한다.
# tf.LSTM
'''
tf.keras.layers.LSTM(
    units, activation='tanh', recurrent_activation='sigmoid',
    use_bias=True, kernel_initializer='glorot_uniform',
    recurrent_initializer='orthogonal',
    bias_initializer='zeros', unit_forget_bias=True,
    kernel_regularizer=None, recurrent_regularizer=None, bias_regularizer=None,
    activity_regularizer=None, kernel_constraint=None, recurrent_constraint=None,
    bias_constraint=None, dropout=0.0, recurrent_dropout=0.0,
    return_sequences=False, return_state=False, go_backwards=False, stateful=False,
    time_major=False, unroll=False, **kwargs
)
'''

question_input = Input(shape = (None,), dtype = 'int32', name = 'question')
# 앞에서 text_input에 대한 처리를 했던 것처럼 question_input도 같이 처리해준다.

embedded_question = layers.Embedding(32, question_vocabulary_size)(question_input)
encoded_question = layers.LSTM(16)(embedded_question)

concatenated = layers.concatenate([encoded_text, encoded_question], axis = -1)
# encoded_question과 encoded_text을 합친다.
# 이 부분이 function API의 가장 큰 특징이다.

answer = layers.Dense(answer_vocabulary_size, activation = 'softmax')(concatenated)
# softmax 분류기를 마지막에 추가한다.

model = Model([text_input, question_input], answer)
# Model(input_tensor, output_tensor)

model.compile(optimizer = 'rmsprop', loss = 'categorical_crossentropy', metrics = ['acc'])

## 7.2 Feeding data to a multi-input model

In [ ]:
import numpy as np

num_samples = 1000
max_length = 100

text = np.random.randint(1, text_vocabulary_size, size = (num_samples, max_length))
# Numpy 데이터를 생성한다. 
# np.randint
'''
random.randint() 함수는 [최소값, 최대값)의 범위에서 임의의 정수를 만듭니다.

np.random.randint(2, size=5)는 [0, 2) 범위에서 다섯개의 임의의 정수를 생성합니다.
np.random.randint(2, 4, size=5)는 [2, 4) 범위에서 다섯개의 임의의 정수를 생성합니다.
np.random.randint(1, 5, size=(2, 3))는 [1, 5) 범위에서 (2, 3) 형태의 어레이를 생성합니다.
'''

question = np.random.randint(1, question_vocabulary_size, size = (num_samples, max_length))
answers = np.random.randint(0,1,size = (num_samples, answer_vocabulary_size))
# answer 벡터는 정수가 아닌 타입으로 인코딩된다.

model.fit([text, question], answers, epochs = 10, batch_size = 128)
# 입력값을 list로 만들어 fitting한다.
model.fit({'text' : text, 'question' : question}, answers, epochs = 10, batch_size = 128)
# 이름과 벡터를 매핑한 형태인 사전을 사용하여 fitting한다.

Epoch 1/10


InvalidArgumentError:  indices[120,0] = 7850 is not in [0, 32)
	 [[node functional_1/embedding_1/embedding_lookup (defined at <ipython-input-2-6d5c44c1ea4b>:21) ]] [Op:__inference_train_function_5783]

Errors may have originated from an input operation.
Input Source operations connected to node functional_1/embedding_1/embedding_lookup:
 functional_1/embedding_1/embedding_lookup/3527 (defined at C:\ProgramData\Anaconda3\envs\myenv\lib\contextlib.py:113)

Function call stack:
train_function


## 7.3 Functional API implementation of a three-output model

In [ ]:
from keras import layers
from keras import Input
from keras.models import Model

vocabulary_size = 50000
num_income_groups = 10

posts_input = Input(shape = (None,), dtype = 'int32', name = 'posts')
embedded_posts = layers.Embedding(256, vocabulary_size)(posts_input)

x = layers.Conv1D(128, 5, activation = 'relu')(embedded_posts)
x = layers.MaxPooling1D(5)(x)

x = layers.Conv1D(256, 5, activation = 'relu')(x)
x = layers.Conv1D(256, 5, activation = 'relu')(x)
x = layers.MaxPooling1D(5)(x)

x = layers.Conv1D(256, 5, activation = 'relu')(x)
x = layers.Conv1D(256, 5, activation = 'relu')(x)
x = layers.GlobalMaxPooling1D()(x)

x = layers.Dense(128, activation = 'relu')(x)

# multi-outputs
age_prediction = layers.Dense(1, name = 'age')(x)
income_prediction = layers.Dense(num_income_groups, activation = 'softmax', name = 'income')(x)
gender_prediction = layers.Dense(1, activation = 'sigmoid', name = 'gender')(x)

model = Model(posts_input, [age_prediction, income_prediction, gender_prediction])

## 7.4 Compilation options of a multi-output model : multiple losses

In [ ]:
model.compile(optimizer = 'rmsprop', loss = {'age' : 'mse', 'income' : 'categorical_crossentropy', 'gender' : 'binary_crossentropy'})
# age는 회귀문제이므로 mse, 수입은 10개로 종류를 나누었으므로 categorical, 성별은 남/여성이므로 binary

'''
model.compile(optimizer = 'rmsprop', loss = {'mse', 'categorical_crossentropy', 'binary_crossentropy'})
'''

"\nmodel.compile(optimizer = 'rmsprop', loss = {'mse', 'categorical_crossentropy', 'binary_crossentropy'})\n"

## 7.5 Compilation options of a multi-output model : loss weighting

In [ ]:
model.compile(optimizer = 'rmsprop', loss = ['mse', 'categorical_crossentropy', 'binary_crossentropy'],
             loss_weights = [0.25, 1., 10,])

## 7.6 Feeding data to a multi-output model

In [ ]:
model.fit(posts, [age_targets, income_targets, gender_targets], 
         epochs = 10, batch_size = 64)

TypeError: int() argument must be a string, a bytes-like object or a number, not 'NoneType'

In [ ]:
from keras import layers

branch_a = layers.Conv2D(128, 1, activation = 'relu', strides = 2)(x)
# 모든 브랜치는 보폭의 크기가 2이다. 2로 맞추어주어 모든 브랜치의 결과값의 사이즈가 같도록 한다.
# for concatenating

branch_b = layers.Conv2D(128, 1, activation = 'relu')(x)
branch_b = layers.Conv2D(128, 3, activation = 'relu', strides = 2)(branch_b)

branch_c = layers.AveragePooling(3, strides = 2)(x)
branch_c = layers.Conv2D(128, 3, activation = 'relu')(branch_c)
# 이 브랜치에는 average 풀링을 한다.

branch_d = layers.Conv2D(128, 1, activation = 'relu')(x)
branch_d = layers.Conv2D(128, 3, activation = 'relu')(branch_d)
branch_d = layers.Conv2D(128, 3, activation = 'relu', strides = 2)(branch_d)

output = layers.concatentate([branch_a, branch_b, branch_c, branch_d], axis = -1)
# 4개의 브랜치를 합친다.

In [ ]:
from keras import layers
from keras import Input
from keras.models import Model

lstm = layers.LSTM(32)

left_input = Input(shape = (None,128))
left_output = lstm(left_input)
# 왼쪽의 브랜치에 대한 모델을 만들었다. 입력값은 변수의 길이의 크기가 128이다

right_input = Input(shape = (None, 128))
right_output = lstm(right_input)
# 오른쪽 브랜치에 대한 모델. 기존에 제작한 레이어를 재사용한다면, 그 레이어의 가중치를 재사용하는 것이다.

merged = layers.concatenate([left_output, right_output], axis = -1)
# left 브랜치의 출력값과 right 브랜치의 출력값을 합쳐서 결과값으로 만들어준다.
predictions = layers.Dense(1, activation = 'sigmoid')(merged)
# concatenate함수를 통해 하나로 합친 결과값을 Dense 분류기에 넣고 손실값을 계산한다.

model = Model([left_input, right_input], predictions)
# 모델을 만들고 학습시킨다. 이러한 모델을 학습시킬 때, LSTM층의 가중치는 두 입력값을 기반으로 하여 수정된다.
model.fit([left_data, right_data], targets)

In [ ]:
from keras import layers
from keras import applications
from keras import Input

xception_base = applications.Xception(weights = None, include_top = False)
# 이미지 가공 모델의 기본 모델은 바로 Xceptino 네트워크이다. (합성곱 층만 존재함)

left_input = Input(shape = (250, 250, 3))
right_input = Input(shape = (250, 250, 3))
# 입력값은 바로 250x250 크기의 칼라 이미지이다.

left_features = xception_base(left_input)
right_input = xception_base(right_input)
# xception_base 모델을 여러 번 사용, 재사용하였다. 

merged_features = layers.concatenate([left_features, right_input], axis = -1)